In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import dateutil

## Don't forget to correlate the 30 IDs from tracefinder with the peaks in CD 3.3 output

In [7]:
####### Animal phenotypes/metadata

animals_used = [1091, 1093, 1060, 1062, 1074, 1092, 1102, 1076, 1082, 1101]
diabetic = [1076, 1082, 1101]
impaired = [1060, 1062, 1074, 1092, 1102]
normal = [1091, 1093]
bg_dict = {'diabetic': diabetic, 'impaired': impaired, 'normal': normal}
animal_to_tolerance_dict = {1076: 'diabetic', 1082: 'diabetic',1101: 'diabetic',
                            1060: 'impaired', 1062: 'impaired', 1074: 'impaired', 1092: 'impaired', 1102: 'impaired', 
                            1091: 'normal', 1093: 'normal',}

obs = pd.read_excel(r'..\data\metadata\20210421_FBGRBGOGTT_Huishi sample sheet.xlsx', sheet_name='cleaned')
obs['animal'] = obs['animal'].str.strip('TM').astype('int')
obs['datetime'] = obs['when'].apply(dateutil.parser.parse)
animal_phenotypes = pd.read_excel(r'..\data\metadata\20210421_FBGRBGOGTT_Huishi sample sheet.xlsx', sheet_name='animal phenotypes')
animal_phenotypes['animal'] = animal_phenotypes['animal'].str.strip('TM').astype('int')


####### Lipidomics data
# fr_filt is the fully processed dataset
# fr_raw = pd.read_csv(r'..\data\search\peak_finder\Final_Results.csv').dropna(axis=1, how='all')
# fr_norm = pd.read_csv(r'..\data\processed\lipidomics_linregress_normalized_data.csv')
fr_filt = pd.read_csv(r'..\data\processed\lipidomics_normalized_filtered_data.csv', index_col=0)
lfg = pd.read_csv(r'..\data\metadata\file_grouping.csv', index_col=0)  # lfg = lipid file grouping
fr_rawfiles = lfg[lfg['quant_file'] == True]['fr_name'].to_list()


####### Metabolomics data from Tracefinder and Compound Discoverer 3.3
mfg = pd.read_csv(r'..\data\metadata\metabolomics_file_grouping.csv', index_col=0)  # mfg = metabolomics file grouping
tf = pd.read_excel(r'..\data\raw\metabolomics\20210904_KAO_Huishi_HILIC_Tracefinder_peakAreas.xlsx', index_col=0)
tf_metab_cols = tf.columns
tf_quant_filenames = mfg[mfg['quant_file'] == True]['File Name'].to_list()
# tf = tf.replace(0, 100)  # REPLACING ZEROES WITH 1E3

metab = pd.read_csv(r'..\data\processed\CD33_metabs_manual_curated.csv')
# Filter rows for only Compounds table by checking for NaN in first column
metab = metab.loc[metab[metab.columns[0]].notna()]
metab = metab.dropna(how='all', axis=1)  # Drop columns that are all NaN

cols = metab.columns
pr_cols = metab.filter(regex='Peak Rating')  # pr == Peak Rating 
pr_sample_cols = pr_cols.filter(regex='BG').columns
pr_sample_cols
rawfile_cols = metab.filter(regex='Group Area').columns
# Rename the columns to remove "Group Area: " from each quant col
renamed_rawfile_cols = {col: col.lstrip('Group Area: ') for col in rawfile_cols}
metab = metab.rename(renamed_rawfile_cols, axis=1)
# sample_cols = metab.filter(regex='Group Area').filter(regex='BG').columns
rawfile_cols = list(renamed_rawfile_cols.values())

# Log2 transform data
metab.loc[:, list(rawfile_cols) + ['Area (Max.)']] = np.log2(metab.loc[:, list(rawfile_cols) + ['Area (Max.)']].astype('float'))  

# During manual curation, BAD features were CHECKED == TRUE.
# So GOOD FEATURES have CHECKED == FALSE
metab = metab[metab['Checked'] == 'FALSE']
metab = metab.reset_index(drop=True)
metab

,20210831_KAO_HILIC_T112M_20210330_8wk_RBG,20210831_KAO_HILIC_T193M20210406_9wk_RBG,20210831_KAO_HILIC_T1060M_20210322_8wk_FBG,20210831_KAO_HILIC_T1060M_20210325_8wk_RBG,20210831_KAO_HILIC_T1060M_20210329_9wk_FBG,20210831_KAO_HILIC_T1060M_20210401_9wk_RBG,20210831_KAO_HILIC_T1060M_20210405_10wk_FBG,20210831_KAO_HILIC_T1060M_20210408_10wk_RBG,20210831_KAO_HILIC_T1062M_20210322_8wk_FBG,20210831_KAO_HILIC_T1062M_20210325_8wk_RBG,...,Peak Rating: 20210831_KAO_HILIC_T1101M_20210330_8wk_RBG.raw (F59),Peak Rating: 20210831_KAO_HILIC_T1101M_20210402_9wk_FBG.raw (F60),Peak Rating: 20210831_KAO_HILIC_T1101M_20210406_9wk_RBG.raw (F61),Peak Rating: 20210831_KAO_HILIC_T1101M_20210410_10wk_FBG.raw (F62),Peak Rating: 20210831_KAO_HILIC_T1101M_20210413_10wk_RBG.raw (F63),Peak Rating: 20210831_KAO_HILIC_T1102M_20210327_8wk_FBG.raw (F64),Peak Rating: 20210831_KAO_HILIC_T1102M_20210402_9wk_FBG.raw (F65),Peak Rating: 20210831_KAO_HILIC_T1102M_20210406_9wk_RBG.raw (F66),Peak Rating: 20210831_KAO_HILIC_T1102M_20210410_10wk_FBG.raw (F67),Peak Rating: 20210831_KAO_HILIC_T1102M_20210413_10wk_RBG.raw (F68)
0,30.624453,30.644187,30.560702,30.610134,30.606110,30.996475,29.787913,30.636811,30.572489,30.636650,...,6.2,5.7,NaN,NaN,6.2,NaN,6.2,NaN,5.7,NaN
1,29.689560,29.774822,29.945500,29.309067,30.082627,29.431175,30.096643,29.270487,30.703658,30.109212,...,8.4,7.8,8.4,9.5,9.5,10,8.9,9.5,8.4,7.8
2,29.931633,29.942531,29.528341,29.827946,29.599444,30.018239,29.253402,29.673409,29.569621,30.189935,...,6.2,8.4,8.9,8.4,7.8,8.4,8.4,7.8,8.4,8.4
3,28.978826,29.128997,29.839308,28.459047,30.235353,28.625995,30.252208,28.265490,30.641931,29.492769,...,8.4,8.4,8.9,8.4,9.5,9.5,9.5,9.5,7.8,7.8
4,24.596731,26.201371,17.148994,17.431244,19.169210,26.312933,27.209158,25.956118,21.774699,26.822243,...,8.4,8.4,9.5,8.4,8.4,NaN,8.9,7.8,8.9,8.4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
700,19.698029,19.891287,17.763246,19.627882,19.473107,18.243526,19.477656,19.609624,19.345424,19.619823,...,7.4,0.3,6.8,0.3,NaN,0.3,NaN,6.8,0.3,6.8
701,20.058637,19.564824,19.261637,19.559839,18.631130,20.407732,19.581830,19.719846,19.717326,19.575585,...,7.4,0.3,5.8,3.4,NaN,7.9,6.8,3,6.8,7.4
702,17.444434,19.490645,19.101047,20.183049,19.537407,16.575568,18.931378,19.504332,19.888199,18.929660,...,NaN,7.4,NaN,NaN,NaN,7.9,0.3,7.9,7.4,NaN
703,19.294282,18.659783,20.034384,19.589771,20.037159,19.774595,20.258819,18.942582,19.854122,18.940518,...,7.9,0.3,NaN,0.3,0.3,0.3,6.3,8.5,7.9,0.3


In [112]:
fg = lfg.rename({'rat_label':'animal'}, axis=1).reset_index()
fg = fg[fg['quant_file'] == True].merge(mfg, 
                    left_on=['animal', 'bg_type', 'week', 'quant_file'],
                    right_on=['animal', 'bg_type', 'week', 'quant_file'],
                    suffixes=('_lipid', '_metab'))
fg['animal'] = fg['animal'].astype('int')
fg['week'] = fg['week'].astype('int')
fg.rename({'analytical_run_order': 'lipid_run_order', 'analytical_order': 'metab_run_order'}, axis=1, inplace=True)
fg['combined_col_name'] = fg['animal'].astype('str') + '_' + fg['week'].astype('str') + '_' + fg['bg_type']
fg.to_csv(r'..\data\metadata\combined_metab_lipid_file_grouping.csv')

## Merged df will have column names with "animal_week_bg"
## Row names will have "m_" for metabolites, and "l_" for lipids

In [28]:
fg

,File Name_lipid,lipid_run_order,fr_name,animal,bg_type,quant_file,week,lipidex_file_number,date,glucose_tolerance,bg,File Name_metab,metab_run_order,fixed_filename,qc_file,blank_file,gluc_tol,combined_col_name
0,20210729_AJ_Toh_RatBloodGlucose_T1076M_2021040...,7,20210729_AJ_Toh_RatBloodGlucose_T1076M_2021040...,1076,FBG,True,10,27,2021-04-05,diabetic,71.000000,20210831_KAO_HILIC_T1076M_20210405_10wk_FBG,21,20210831_KAO_HILIC_T1076M_20210405_10wk_FBG,False,False,diabetic,1076_10_FBG
1,20210729_AJ_Toh_RatBloodGlucose_T1062M_2021032...,8,20210729_AJ_Toh_RatBloodGlucose_T1062M_2021032...,1062,FBG,True,8,9,2021-03-22,impaired,52.333333,20210831_KAO_HILIC_T1062M_20210322_8wk_FBG,61,20210831_KAO_HILIC_T1062M_20210322_8wk_FBG,False,False,impaired,1062_8_FBG
2,20210729_AJ_Toh_RatBloodGlucose_T1093M_2021040...,9,20210729_AJ_Toh_RatBloodGlucose_T1093M_2021040...,1093,FBG,True,9,49,2021-04-02,normal,64.000000,20210831_KAO_HILIC_T1093M_20210402_9wk_FBG,50,20210831_KAO_HILIC_T1093M_20210402_9wk_FBG,False,False,normal,1093_9_FBG
3,20210729_AJ_Toh_RatBloodGlucose_T1060M_2021032...,10,20210729_AJ_Toh_RatBloodGlucose_T1060M_2021032...,1060,RBG,True,8,4,2021-03-25,impaired,225.000000,20210831_KAO_HILIC_T1060M_20210325_8wk_RBG,59,20210831_KAO_HILIC_T1060M_20210325_8wk_RBG,False,False,impaired,1060_8_RBG
4,20210729_AJ_Toh_RatBloodGlucose_T1091M_2021041...,11,20210729_AJ_Toh_RatBloodGlucose_T1091M_2021041...,1091,FBG,True,10,39,2021-04-10,normal,48.500000,20210831_KAO_HILIC_T1091M_20210410_10wk_FBG,56,20210831_KAO_HILIC_T1091M_20210410_10wk_FBG,False,False,normal,1091_10_FBG
5,20210729_AJ_Toh_RatBloodGlucose_T1092M_2021040...,12,20210729_AJ_Toh_RatBloodGlucose_T1092M_2021040...,1092,RBG,True,9,44,2021-04-06,impaired,70.500000,20210831_KAO_HILIC_T1092M_20210406_9wk_RBG,18,20210831_KAO_HILIC_T1092M_20210406_9wk_RBG,False,False,impaired,1092_9_RBG
6,20210729_AJ_Toh_RatBloodGlucose_T1093M_2021041...,13,20210729_AJ_Toh_RatBloodGlucose_T1093M_2021041...,1093,RBG,True,10,52,2021-04-13,normal,69.500000,20210831_KAO_HILIC_T1093M_20210413_10wk_RBG,45,20210831_KAO_HILIC_T1093M_20210413_10wk_RBG,False,False,normal,1093_10_RBG
7,20210729_AJ_Toh_RatBloodGlucose_T1082M_2021040...,14,20210729_AJ_Toh_RatBloodGlucose_T1082M_2021040...,1082,FBG,True,10,33,2021-04-05,diabetic,50.000000,20210831_KAO_HILIC_T1082M_20210405_10wk_FBG,41,20210831_KAO_HILIC_T1082M_20210405_10wk_FBG,False,False,diabetic,1082_10_FBG
8,20210729_AJ_Toh_RatBloodGlucose_T1060M_2021040...,15,20210729_AJ_Toh_RatBloodGlucose_T1060M_2021040...,1060,RBG,True,9,6,2021-04-01,impaired,83.000000,20210831_KAO_HILIC_T1060M_20210401_9wk_RBG,5,20210831_KAO_HILIC_T1060M_20210401_9wk_RBG,False,False,impaired,1060_9_RBG
9,20210729_AJ_Toh_RatBloodGlucose_T1101M_2021040...,16,20210729_AJ_Toh_RatBloodGlucose_T1101M_2021040...,1101,FBG,True,9,55,2021-04-02,diabetic,43.000000,20210831_KAO_HILIC_T1101M_20210402_9wk_FBG,44,20210831_KAO_HILIC_T1101M_20210402_9wk_FBG,False,False,diabetic,1101_9_FBG


In [44]:
fr_filt['new_index'] =  ['l_' + str(x) for x in range(fr_filt.shape[0])]
fr_filt['new_index']

Unique Identifier
unknown_679.05463_0.536         l_0
unknown_641.1087_0.537          l_1
unknown_951.15973_0.541         l_2
unknown_329.03067_0.542         l_3
unknown_969.13147_0.544         l_4
                              ...  
unknown_897.82532_20.148     l_1024
unknown_974.9118_20.163      l_1025
unknown_593.58716_20.175     l_1026
unknown_871.80945_20.196     l_1027
unknown_1003.93829_20.435    l_1028
Name: new_index, Length: 1029, dtype: object

In [36]:
metab['new_index'] = 'm_' + metab.index.astype('str')
metab['new_index']

0        m_0
1        m_1
2        m_2
3        m_3
4        m_4
       ...  
700    m_700
701    m_701
702    m_702
703    m_703
704    m_704
Name: new_index, Length: 705, dtype: object

In [89]:
metab_rename = fg[['File Name_metab', 'combined_col_name']].set_index('File Name_metab').T.to_dict('records')[0]
lipid_rename = fg[['fr_name', 'combined_col_name']].set_index('fr_name').T.to_dict('records')[0]

In [103]:
new_lipid_df = fr_filt[fr_rawfiles + ['new_index']]
new_lipid_df.set_index('new_index', inplace=True)
new_lipid_df.rename(lipid_rename, axis=1, inplace=True)
new_lipid_df = np.log2(new_lipid_df)
new_lipid_df

C:\Users\bjanderson23\anaconda3\lib\site-packages\pandas\core\frame.py:4449: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


,1076_10_FBG,1062_8_FBG,1093_9_FBG,1060_8_RBG,1091_10_FBG,1092_9_RBG,1093_10_RBG,1082_10_FBG,1060_9_RBG,1101_9_FBG,...,1092_10_FBG,1082_8_RBG,1062_8_RBG,1102_10_FBG,1093_8_RBG,1074_9_FBG,1062_9_RBG,1101_8_RBG,1092_8_RBG,1092_10_RBG
new_index,,,,,,,,,,,,,,,,,,,,,
l_0,20.405355,23.313111,23.218699,24.012349,24.302356,23.901455,23.883301,23.387975,24.421692,23.463756,...,22.462249,23.360916,22.399150,23.163419,22.737840,22.116533,23.352451,23.527438,23.858693,23.617549
l_1,21.533648,24.586227,24.375151,25.685664,25.422294,24.936916,24.779941,24.696276,25.054941,24.934805,...,24.663849,23.700392,23.382317,24.385568,23.789670,22.248759,22.513249,24.558075,24.818525,24.575904
l_2,20.665848,23.731244,23.038733,24.916555,24.042666,24.162971,23.627979,24.085858,23.169508,24.010849,...,23.452626,22.549975,23.249352,23.680978,23.989589,23.527507,24.002570,23.886203,24.069137,23.455789
l_3,24.400930,26.811664,26.708762,25.977120,27.313654,27.337061,27.415597,27.230430,27.598719,27.126653,...,27.210668,26.194952,26.911119,26.805997,26.145097,26.638404,27.217392,26.970428,27.175265,26.482705
l_4,19.378685,22.709669,21.722467,23.985839,22.321161,23.184906,22.795191,22.878785,22.397716,22.703282,...,22.602208,21.905509,22.226570,22.485669,22.208159,22.236114,23.151359,22.823820,21.750593,22.047030
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
l_1024,18.897008,19.348489,21.529536,17.921098,20.133479,17.824376,18.836779,19.255315,18.261890,19.685160,...,18.604841,18.755897,18.996482,20.095162,17.759520,18.908638,18.093365,17.607114,17.326455,16.927718
l_1025,16.810111,17.364343,20.099612,18.430542,17.986741,18.099621,18.145468,14.496938,18.854601,18.324624,...,16.326408,19.046476,18.653201,18.635659,17.382575,17.575165,18.798950,16.704255,18.420810,17.646532
l_1026,17.724121,19.240306,21.430516,17.488155,19.996615,17.383197,18.282680,19.507853,17.718258,19.526233,...,18.815858,18.693961,18.778798,20.192346,17.113028,18.925197,17.773412,17.541022,16.974304,16.687194


In [104]:
new_metab_df = metab[rawfile_cols + ['new_index']]
new_metab_df.set_index('new_index', inplace=True)
new_metab_df.rename(metab_rename, axis=1, inplace=True)
new_metab_df

C:\Users\bjanderson23\anaconda3\lib\site-packages\pandas\core\frame.py:4449: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


,1102_8_RBG,1093_9_RBG,1060_8_FBG,1060_8_RBG,1060_9_FBG,1060_9_RBG,1060_10_FBG,1060_10_RBG,1062_8_FBG,1062_8_RBG,...,1101_8_RBG,1101_9_FBG,1101_9_RBG,1101_10_FBG,1101_10_RBG,1102_8_FBG,1102_9_FBG,1102_9_RBG,1102_10_FBG,1102_10_RBG
new_index,,,,,,,,,,,,,,,,,,,,,
m_0,30.624453,30.644187,30.560702,30.610134,30.606110,30.996475,29.787913,30.636811,30.572489,30.636650,...,30.612707,30.672859,30.606863,30.648334,30.759784,30.588007,30.704747,30.585579,30.699123,30.458963
m_1,29.689560,29.774822,29.945500,29.309067,30.082627,29.431175,30.096643,29.270487,30.703658,30.109212,...,29.830917,30.044828,29.781465,29.816131,29.925683,30.314288,30.159520,30.126519,30.203473,29.573941
m_2,29.931633,29.942531,29.528341,29.827946,29.599444,30.018239,29.253402,29.673409,29.569621,30.189935,...,29.933457,29.620839,29.637645,29.693668,29.755497,29.737910,30.090489,29.752281,29.744298,29.673058
m_3,28.978826,29.128997,29.839308,28.459047,30.235353,28.625995,30.252208,28.265490,30.641931,29.492769,...,29.104520,29.974868,28.941815,29.634254,28.991867,30.068836,29.869304,29.589192,29.923218,28.952127
m_4,24.596731,26.201371,17.148994,17.431244,19.169210,26.312933,27.209158,25.956118,21.774699,26.822243,...,25.684321,26.709748,27.729752,27.103362,26.606979,19.180453,24.860989,27.789178,26.471452,25.842127
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
m_700,19.698029,19.891287,17.763246,19.627882,19.473107,18.243526,19.477656,19.609624,19.345424,19.619823,...,19.672762,19.406261,19.583848,19.234988,19.294171,19.309877,19.446901,19.521370,19.818652,19.496501
m_701,20.058637,19.564824,19.261637,19.559839,18.631130,20.407732,19.581830,19.719846,19.717326,19.575585,...,19.596183,19.285396,19.835033,20.170272,19.634797,19.464967,19.607444,19.928946,19.760808,19.745445
m_702,17.444434,19.490645,19.101047,20.183049,19.537407,16.575568,18.931378,19.504332,19.888199,18.929660,...,18.904319,20.001674,18.553910,19.610188,18.053380,19.653173,19.493393,19.415631,19.841179,18.860218


In [106]:
merged = pd.concat([new_metab_df, new_lipid_df])
merged.to_csv(r'..\data\processed\metab_lipid_merged_df.csv', )

In [109]:
pd.read_csv(r'..\data\processed\metab_lipid_merged_df.csv', index_col=0)

,1102_8_RBG,1093_9_RBG,1060_8_FBG,1060_8_RBG,1060_9_FBG,1060_9_RBG,1060_10_FBG,1060_10_RBG,1062_8_FBG,1062_8_RBG,...,1101_8_RBG,1101_9_FBG,1101_9_RBG,1101_10_FBG,1101_10_RBG,1102_8_FBG,1102_9_FBG,1102_9_RBG,1102_10_FBG,1102_10_RBG
new_index,,,,,,,,,,,,,,,,,,,,,
m_0,30.624453,30.644187,30.560702,30.610134,30.606110,30.996475,29.787913,30.636811,30.572489,30.636650,...,30.612707,30.672859,30.606863,30.648334,30.759784,30.588007,30.704747,30.585579,30.699123,30.458963
m_1,29.689560,29.774822,29.945500,29.309067,30.082627,29.431175,30.096643,29.270487,30.703658,30.109212,...,29.830917,30.044828,29.781465,29.816131,29.925683,30.314288,30.159520,30.126519,30.203473,29.573941
m_2,29.931633,29.942531,29.528341,29.827946,29.599444,30.018239,29.253402,29.673409,29.569621,30.189935,...,29.933457,29.620839,29.637645,29.693668,29.755497,29.737910,30.090489,29.752281,29.744298,29.673058
m_3,28.978826,29.128997,29.839308,28.459047,30.235353,28.625995,30.252208,28.265490,30.641931,29.492769,...,29.104520,29.974868,28.941815,29.634254,28.991867,30.068836,29.869304,29.589192,29.923218,28.952127
m_4,24.596731,26.201371,17.148994,17.431244,19.169210,26.312933,27.209158,25.956118,21.774699,26.822243,...,25.684321,26.709748,27.729752,27.103362,26.606979,19.180453,24.860989,27.789178,26.471452,25.842127
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
l_1024,18.104875,18.387917,20.517490,17.921098,20.671826,18.261890,19.568571,17.211559,19.348489,18.996482,...,17.607114,19.685160,18.426683,18.108585,18.174690,19.778597,19.199924,19.205373,20.095162,17.314487
l_1025,18.821648,19.088279,19.164463,18.430542,19.113894,18.854601,17.625169,18.154900,17.364343,18.653201,...,16.704255,18.324624,18.699032,16.629913,18.468857,18.207822,17.909403,19.735550,18.635659,19.182523
l_1026,17.878105,17.974406,20.603688,17.488155,20.788152,17.718258,19.769735,17.016673,19.240306,18.778798,...,17.541022,19.526233,18.163185,18.406753,17.767814,19.924372,19.372148,18.938731,20.192346,17.454831
